# MODIS DOWLOADING FROM Earth Enginee

In [1]:
import geemap
import ee
# Trigger the authentication flow.
ee.Authenticate()
# Initialize the library.
ee.Initialize()

C:\Users\qle\AppData\Local\miniconda3\envs\text\Lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\qle\AppData\Local\miniconda3\envs\text\Lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
C:\Users\qle\AppData\Local\miniconda3\envs\text\Lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


Enter verification code:  4/1Adeu5BVLfFHjywf_i04uksRtGH3wRkxLj5vLR2xPOKjKO46619ERo9HRh_E



Successfully saved authorization token.


In [2]:
#Pandas modules to interact with spatial data
import geopandas as gpd
import pandas as pd
from geopandas import GeoDataFrame
from shapely.geometry import Point,Polygon
from geemap import geopandas_to_ee
import shapely.wkt

from shapely.geometry import mapping
from pygeosys.util.dataframe import chunk_dataframe
from generate_data_ee import *
import numpy as np
from functools import reduce

In [7]:
dataPath = 'CSV_Data/'
file = 'AllDrought.csv'
# file = 'SoyExRains.csv'
# file = 'SoyExRain_Non.csv'

usecols = ['sfd_id','WKT']
rename = {
            'WKT': 'geometry'
}
data = pd.read_csv(dataPath+file,
                   sep=';',
                   usecols=usecols
                  )
data = data[usecols]
data.rename(columns=rename, inplace=True)
# to check the index from interupting process
data[data['sfd_id'] == 104838039 ]
data = data[72:73]
data

,sfd_id,geometry
72,104838039,"MULTIPOLYGON (((-54.15842318 -25.22023698,-54...."


In [8]:
### Parameters
'''
#year=2021
year=2020
date_start = str(year)+'-10-01' # fall tillage : 08-01 to 11-01 ; spring tillage : 02-01 to 05-01
date_end = str(year)+'-12-31'
#''
#year=2022
year=2021
date_start = str(year)+'-01-01'
date_end = str(year)+'-02-15'
'''
year=2019
date_start = str(year)+'-01-01'
date_end = str(year)+'-12-31'

date_start

'2019-01-01'

In [7]:
'''
from shapely.geometry import Polygon
# AOI polygon 
# Tyope this below
polygon = ee.Geometry.Polygon(
    [[[1.443458, 43.605878], [1.441913, 43.602366], [1.447449, 43.598388], [1.455345, 43.602957], [1.452212, 43.607525], [1.443458, 43.605878]]],None,False
)
polyFeature = ee.Feature(polygon, {'foo': 42, 'bar': 'tart'})
Map.addLayer(polyFeature, {} , 'feature')
#Map
'''

"\nfrom shapely.geometry import Polygon\n# AOI polygon \n# Tyope this below\npolygon = ee.Geometry.Polygon(\n    [[[1.443458, 43.605878], [1.441913, 43.602366], [1.447449, 43.598388], [1.455345, 43.602957], [1.452212, 43.607525], [1.443458, 43.605878]]],None,False\n)\npolyFeature = ee.Feature(polygon, {'foo': 42, 'bar': 'tart'})\nMap.addLayer(polyFeature, {} , 'feature')\n#Map\n"

# MODIS Single Dowload Checking

In [21]:
for idx,row in data.iterrows():        
    # Convert to shapely file
    coordinate_raw = shapely.wkt.loads(row['geometry'])
    # Get the coordinates
    coordinate = mapping(coordinate_raw).get('coordinates')[0]
    polygon = ee.Geometry.Polygon(coordinate,None,False)

In [30]:
#CHECKING The number of images before dowloading
im_coll = get_MODIS_col(polygon,date_start,date_end) 
im_coll = im_coll.map(NDVI_MODIS)#.map(add_cld_shdw_mask).map(apply_cld_shdw_mask)#
# For multiple downloading SAR image Red/Blue/Green
imList=im_coll.select('Nadir_Reflectance_Band1','Nadir_Reflectance_Band3','Nadir_Reflectance_Band4').toList(im_coll.size())        
#imList=im_coll.select('NDVI').toList(im_coll.size())
imList

In [18]:
im_coll.aggregate_array('relativeOrbitNumber_start').getInfo()
print('None')

None


In [19]:
## get available dates 
import time
acq_times = im_coll.aggregate_array('system:time_start').getInfo()
len([time.strftime('%x', time.gmtime(acq_time/1000)) for acq_time in acq_times])

56

In [21]:
img = ee.Image(im_coll.select('VV','VH').first())#get first image of collection 
#img = ee.Image(im_coll.get(0))#get first image of collection 
#img = ee.Image(im_coll.select('VV','VH').filterDate(ee.Date(date_start),ee.Date(date_end)))
#img = ee.Image(im_coll.get(1))#get first image of collection 
img
type(img)

ee.image.Image

In [22]:
task=ee.batch.Export.image.toCloudStorage(**{
          'image': img.clip(polygon),
          'description': 'Toulouse SAR',
          'bucket': 'datascience-sandbox-dev', #bucket name never changes
          'fileNamePrefix':'LE/ChangeDetection/SAR/Toulouse' ,#path to file
          'scale': 10,#resolution
          'region':polygon,
          'maxPixels': 15000000000,
          'skipEmptyTiles': True
          })
task.start()

In [20]:
# Get the list of image
im_list = im_coll.toList(im_coll.size())# 56 item
# Take the 1st image
img=im_list.get(0)
img

In [12]:
# For multiple downloading SAR image
imList=im_coll.select('VV','VH').toList(im_coll.size())
for im in range(imList.length().getInfo()):
    img = ee.Image(imList.get(im))
    date = ee.Date(img.get('system:time_start'))
    date2 = date.format('YYYYMMdd').getInfo()
    task=ee.batch.Export.image.toCloudStorage(**{ 'image': img.clip(polygon), 
                                                 'description':'SAR_'+str(row['sfd_id'])+'_'+str(date2), 
                                                 'bucket': 'datascience-sandbox-dev', #bucket name never changes 
                                                 'fileNamePrefix':'LE/ChangeDetection/SAR/'+str(row['sfd_id'])+'/'+ str(row['sfd_id'])+'_Bands_'+str(date2)+'_S1_GRD' , #path to file
                                                 'scale': 10,#resolution 
                                                 'region':polygon, 
                                                 'maxPixels': 15000000000, 
                                                 'skipEmptyTiles': True }) 
    task.start()

# MODIS Multidowloading

In [9]:
def get_MODIS_col(aoi, date_start, date_end):
    '''
    Get data from dataset 'MODIS/061/MCD43A2'
    See more here: https://developers.google.com/earth-engine/datasets/catalog/MODIS_061_MCD43A4#description -> 'MODIS/061/MCD43A4'
    https://developers.google.com/earth-engine/datasets/catalog/MODIS_061_MOD09GA#description -> 'MODIS/061/MOD09GA'
    https://developers.google.com/earth-engine/datasets/catalog/MODIS_MOD09GA_006_NDVI#description -> 'MODIS/MOD09GA_006_NDVI'
    Band select frin vabd 1 ti vabd 7 and BRDF albedo mandatory quality band
    See the band coressponding to the wavelength
    https://www.earthdatascience.org/courses/use-data-open-source-python/multispectral-remote-sensing/modis-data-in-python/
    '''
    im_coll = (ee.ImageCollection('MODIS/MOD09GA_006_NDVI').
                      filterBounds(aoi).
                      filterDate(ee.Date(date_start),ee.Date(date_end))                                      
              )
                      
    return im_coll

def NDVI_MODIS(image):
    # Spatial Resolution 500 m
    return image.addBands(image.normalizedDifference(['sur_refl_b02', 'sur_refl_b01']).rename('NDVI'))

def NDVI_Chanel_Lists(im_coll):
    im_coll = im_coll.map(RVI)
    # For multiple downloading SAR image
    imList = im_coll.select('VV','VH').toList(im_coll.size())
    return imList

def MODIS_ImagesDowloading(dataframe, date_start,date_end):
    for idx,row in dataframe.iterrows():        
        # Convert to shapely file
        coordinate_raw = shapely.wkt.loads(row['geometry'])
        # Get the coordinates
        coordinate = mapping(coordinate_raw).get('coordinates')[0]
        polygon = ee.Geometry.Polygon(coordinate,None,False)
        im_coll = get_MODIS_col(polygon,date_start,date_end) 
        #im_coll = im_coll.map(NDVI_MODIS)#.map(add_cld_shdw_mask).map(apply_cld_shdw_mask)
        # For multiple downloading SAR image Red/Blue/Green
        #imList=im_coll.select('Nadir_Reflectance_Band1','Nadir_Reflectance_Band3','Nadir_Reflectance_Band4').toList(im_coll.size())        
        imList=im_coll.select('NDVI').toList(im_coll.size())
        for im in range(imList.length().getInfo()):
            img = ee.Image(imList.get(im))
            date = ee.Date(img.get('system:time_start'))
            year =date.format('YYYY').getInfo()
            date2 = date.format('YYYYMMdd').getInfo()
            task=ee.batch.Export.image.toCloudStorage(**{ 'image': img.clip(polygon), 
                                                         'description':'MODIS_'+str(row['sfd_id'])+'_'+str(date2), 
                                                         'bucket': 'datascience-sandbox-dev', #bucket name never changes 
                                                         'fileNamePrefix':'LE/ChangeDetection/MODIS/'+str(row['sfd_id'])+'/'+str(year)+'/'+str(row['sfd_id'])+'_Bands_'+str(date2)+'_MODIS' , #path to file
                                                         'scale': 10,#resolution 10, 25 or 40
                                                         'region':polygon, 
                                                         'maxPixels': 15000000000, 
                                                         'skipEmptyTiles': True }) 
            task.start()

In [14]:
MODIS_ImagesDowloading(data,date_start,date_end)